In [2]:
from openai import OpenAI
import os
import time

api_key = os.getenv("OPENAI_API_KEY")
client = OpenAI(api_key=api_key)

Upload files to OpenAI

In [3]:
file = client.files.create(
    file = open("text_library/ThinkingBasketball_202306-202401.txt", "rb"),
    purpose = "assistants"
)
print(file)

FileObject(id='file-Ynjk9fq3wxZG7LJjIZ2vvySY', bytes=381203, created_at=1706618503, filename='ThinkingBasketball_202306-202401.txt', object='file', purpose='assistants', status='processed', status_details=None)


Create Assistant

In [4]:
assistant = client.beta.assistants.create(
    name = "NBA Podcast Intelligence GPT",
    instructions = "You are an expert in basketball / NBA sport analytics, social media content analysis. You are also expert in producing market research reports based on the transcript files in your knowledge base.",
    tools = [{"type": "retrieval"}],
    model = "gpt-4-0125-preview",
    file_ids =  [file.id]
)
print(f"File ID: {file.id}")
print(f"Assistand ID: {assistant.id}")

File ID: file-Ynjk9fq3wxZG7LJjIZ2vvySY
Assistand ID: asst_Jf6bGHwnyaSC0UF14MY6jUTD


Create a thread

In [6]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_aVz4jNAaZxufnu0vTWDMIqgW', created_at=1706618965, metadata={}, object='thread')


Create message

In [8]:
message = client.beta.threads.messages.create(
    thread_id = thread.id,
    role = "user",
    content = "Give me a concise executive summary of the narratives, storylines, and themes in the NBA discussed by Thinking Basketball. Include key stats they mention, at least one per narrative/storyline. Then finish with a concise summary of players and teams mentioned."
)

Run Assistant

In [14]:
run = client.beta.threads.runs.create(
    thread_id = thread.id,
    assistant_id = assistant.id
)
print("Run created! 🚀")
while run.status not in ["completed", "failed"]:
    run = client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id)
    print(f"Run Status: {run.status}")
    if run.status == "failed":
        print(f"Run Failed.")
        break
    time.sleep(3)
else:
    print(f"Run Completed! ✅")

Run created! 🚀
Run Status: queued
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: completed
Run Completed! ✅


Display Response

In [15]:
run.status

'completed'

In [16]:
messages = client.beta.threads.messages.list(thread_id=thread.id)
print(f"Messages: {messages}")
for message in reversed(messages.data):
    print(message.role + ": " + message.content[0].text.value)

Messages: SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_0Cj1d823n6wQ23tUv2yH9MXj', assistant_id='asst_Jf6bGHwnyaSC0UF14MY6jUTD', content=[MessageContentText(text=Text(annotations=[], value="### Executive Summary: NBA Narratives, Storylines, and Themes Discussed by Thinking Basketball\n\n**Narratives/Storylines/Themes:**\n\n- **Ja Morant's Spectacular Season Start:** Thinking Basketball highlights Ja Morant's explosive start to the NBA season, praising his exceptional ball-handling, strength, offensive package, and ability to manipulate passes mid-air. However, concerns about his shooting efficiency and decision-making when driving to the basket were raised, indicating room for improvement to elevate his overall impact further.\n\n- **The Complicated Rudy Gobert Trade:** The narrative around the Minnesota Timberwolves' trade for Rudy Gobert was discussed with a critical lens, highlighting dissatisfaction with the team's dynamics and its implications for the Timberwolves' fut

In [22]:
new_thread = client.beta.threads.create()
print(new_thread)

new_message = client.beta.threads.messages.create(
    thread_id = new_thread.id,
    role = "user",
    content = "give me 5 examples for Thinking Basketball video titles"
)

Thread(id='thread_hdVmY9EusJU3MCbumKaIOVis', created_at=1706621089, metadata={}, object='thread')


In [23]:
run = client.beta.threads.runs.create(
    thread_id = new_thread.id,
    assistant_id = assistant.id
)
print("Run created! 🚀")
while run.status not in ["completed", "failed"]:
    run = client.beta.threads.runs.retrieve(thread_id=new_thread.id, run_id=run.id)
    print(f"Run Status: {run.status}")
    if run.status == "failed":
        print(f"Run Failed.")
        break
    time.sleep(3)
else:
    print(f"Run Completed! ✅")

Run created! 🚀
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: in_progress
Run Status: completed
Run Completed! ✅


In [24]:
new_messages = client.beta.threads.messages.list(thread_id=new_thread.id)
print(f"Messages: {new_messages}")
for message in reversed(new_messages.data):
    print(message.role + ": " + message.content[0].text.value)

Messages: SyncCursorPage[ThreadMessage](data=[ThreadMessage(id='msg_dxe38dZWXZd1pilsO8aFTDJ7', assistant_id='asst_Jf6bGHwnyaSC0UF14MY6jUTD', content=[MessageContentText(text=Text(annotations=[], value="Here are five examples of video titles from the Thinking Basketball channel:\n\n1. The Lakers won the first ever NBA cup by shutting down the league's best offense, executing a perfect game plan【5†source】.\n2. How the Pacers create great shots in transition because of their pace【5†source】.\n3. The strategy of pick up Halliburton full court to slow him down【5†source】.\n4. How LeBron James switches out, moves his feet, and funnels the ball into traffic for a block【5†source】.\n5. A masterful Laker game plan cooked up by Coach Darvin Ham, built around Davis's versatility【5†source】.\n\nNote: The titles have been crafted based on the content description provided in the document for ease of understanding, as precise video titles were not directly mentioned."), type='text')], created_at=17066211